In [1]:
%pip install qdrant-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

import numpy as np
from iris.config.data_pipeline_config_manager import DataPipelineConfigManager
from iris.data_pipeline.qdrant_manager import QdrantManager
from qdrant_client.models import Distance

In [3]:
# Initialize config and manager
qdrant_config = DataPipelineConfigManager().qdrant_config
qdrant_manager = QdrantManager(qdrant_config)

# Connect using context manager
with qdrant_manager as qm:
    print("Available collections:", qm.get_collections())

Available collections: ['iris_dev_nikolaj_storm_texts', 'iris_dev_nikolaj_storm_images']


In [4]:
# Create sample vectors
def create_sample_vectors(vector_size: int, count: int):
    vectors = [np.random.rand(vector_size).astype(np.float32) for _ in range(count)]
    return vectors

# Create collections with sample data
with qdrant_manager as qm:
    # Image vectors (typically 512 dimensions for CLIP)
    image_vectors = create_sample_vectors(vector_size=512, count=3)
    image_payloads = [
        {"source": "sample1.jpg", "category": "dress"},
        {"source": "sample2.jpg", "category": "shirt"},
        {"source": "sample3.jpg", "category": "pants"}
    ]
    
    # Text vectors (same dimensionality as images for CLIP)
    text_vectors = create_sample_vectors(vector_size=512, count=2)
    text_payloads = [
        {"text": "A beautiful summer dress", "type": "description"},
        {"text": "Elegant evening wear", "type": "description"}
    ]
    
    # Create or recreate collections
    qm.create_collection(qdrant_config.image_collection, 512, Distance.COSINE, recreate=True)
    qm.create_collection(qdrant_config.text_collection, 512, Distance.COSINE, recreate=True)
    
    # Insert vectors
    qm.upsert_points(qdrant_config.image_collection, image_vectors, image_payloads)
    qm.upsert_points(qdrant_config.text_collection, text_vectors, text_payloads)
    
    print("Collections after insertion:", qm.get_collections())

Collections after insertion: ['iris_dev_nikolaj_storm_texts', 'iris_dev_nikolaj_storm_images']


In [7]:
# Search examples
with qdrant_manager as qm:
    # Search in image collection using random query vector
    query_vector = np.random.rand(512).astype(np.float32)  # Same size as our collection vectors
    
    # Search images
    print("Searching image collection:")
    image_results = qm.search_points(qdrant_config.image_collection, query_vector, limit=2)
    for result in image_results:
        print(f"Image {result.id}: score={result.score:.3f}, category={result.payload['category']}")
    
    # Search texts
    print("\nSearching text collection:")
    text_results = qm.search_points(qdrant_config.text_collection, query_vector, limit=2)
    for result in text_results:
        print(f"Text {result.id}: score={result.score:.3f}, text='{result.payload['text']}'")

    # Retrieve full vector data for closest match
    if image_results:
        closest_match = qm.retrieve_points(qdrant_config.image_collection, [image_results[0].id])[0]
        print(f"\nClosest image match details:")
        print(f"ID: {closest_match.id}")
        print(f"Source: {closest_match.payload['source']}")
        print(f"Vector shape: {np.array(closest_match.vector).shape}")

Searching image collection:
Image 0: score=0.767, category=dress
Image 1: score=0.762, category=shirt

Searching text collection:
Image 0: score=0.767, category=dress
Image 1: score=0.762, category=shirt

Searching text collection:
Text 1: score=0.755, text='Elegant evening wear'
Text 0: score=0.735, text='A beautiful summer dress'
Text 1: score=0.755, text='Elegant evening wear'
Text 0: score=0.735, text='A beautiful summer dress'

Closest image match details:
ID: 0
Source: sample1.jpg
Vector shape: (512,)

Closest image match details:
ID: 0
Source: sample1.jpg
Vector shape: (512,)


# Accessing Qdrant Cloud GUI
You can view and interact with your vectors through the Qdrant Cloud Console:
1. Go to https://cloud.qdrant.io/
2. Login with your credentials
3. Select your cluster
4. Navigate to the "Collections" tab to see your `test_collection`

In [6]:
# Print direct URL to Qdrant dashboard
dashboard_url = f"{qdrant_config.url}/dashboard"
print(f"View your vectors in Qdrant Dashboard:\n{dashboard_url}")

# For reference, also print the full connection URL
print(f"\nCurrent connection URL:\n{qdrant_config.url}")

View your vectors in Qdrant Dashboard:
https://faa8f550-ff00-4890-b04e-1151b28d57a4.eu-west-2-0.aws.cloud.qdrant.io:6333/dashboard

Current connection URL:
https://faa8f550-ff00-4890-b04e-1151b28d57a4.eu-west-2-0.aws.cloud.qdrant.io:6333
